# Part 1
Loading imports

In [1]:
#imports
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import numpy as np

Retriving the html of the url with the table

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(url)
htmltext = response.content
print(htmltext)

soup = BeautifulSoup(htmltext, 'html.parser')

b'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of postal codes of Canada: M - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"X@HFwQpAAEEAAAvkm@0AAABY","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":995657573,"wgRevisionId":995657573,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Communications in Ontario","Postal

Extract the table object and the rows of the table

In [3]:
table_element = soup.table
rows_elements = table_element.find_all('tr')

Saving the table object in a csv file

In [4]:
file_path = "toronto_neighborhood_wikipedia_df_html_data_to_csv.csv"
File = open(file_path, 'wt+')
Data = csv.writer(File)
try:
    for row in rows_elements:
        FilteredRow = []
        for cell in row.find_all(['td', 'th']):
            FilteredRow.append(cell.get_text().rstrip())
        Data.writerow(FilteredRow)
finally:
    File.close()

Retriving th csv file created before

In [5]:
toronto_neighborhood_df = pd.read_csv(file_path)
toronto_neighborhood_df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Making a count to know how many of `Not assigned` Borough we have

In [6]:
toronto_neighborhood_df.groupby(['Borough'])['Borough'].count()

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Not assigned        77
Scarborough         17
West Toronto         6
York                 5
Name: Borough, dtype: int64

Filtering out the `Not assigned` Borough and checking if we don't have `Not assigned` on Borough any more

In [7]:
toronto_neighborhood_df_dropedna = toronto_neighborhood_df[toronto_neighborhood_df['Borough']!='Not assigned']
toronto_neighborhood_df_dropedna.groupby(['Borough'])['Borough'].count()

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Scarborough         17
West Toronto         6
York                 5
Name: Borough, dtype: int64

Checking for `Not assigned` on Neighbourhood

In [8]:
toronto_neighborhood_df_dropedna.groupby(['Neighbourhood'])['Neighbourhood'].count()

Neighbourhood
Agincourt                                          1
Alderwood, Long Branch                             1
Bathurst Manor, Wilson Heights, Downsview North    1
Bayview Village                                    1
Bedford Park, Lawrence Manor East                  1
                                                  ..
Willowdale, Willowdale West                        1
Woburn                                             1
Woodbine Heights                                   1
York Mills West                                    1
York Mills, Silver Hills                           1
Name: Neighbourhood, Length: 99, dtype: int64

Since I couldn't see if we have `Not assigned` on Neighbourhood checking directly on the data frame

In [9]:
toronto_neighborhood_df_dropedna[toronto_neighborhood_df_dropedna['Neighbourhood']=='Not assigned'].head()

,Postal Code,Borough,Neighbourhood


Since theresn't any `Not assigned` on Neighbourhood printing the shape of the data frame

In [10]:
toronto_neighborhood_df_dropedna.shape

(103, 3)

# Part 2

Load csv file from url

In [11]:
df_csv_postal_code = pd.read_csv("http://cocl.us/Geospatial_data")
df_csv_postal_code.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging csv data with the main data frame

In [12]:
toronoto_neighborhood_postal_code_joined_df = toronto_neighborhood_df_dropedna.join(df_csv_postal_code.set_index('Postal Code'), on='Postal Code')
toronoto_neighborhood_postal_code_joined_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Part 3

In [13]:
from geopy.geocoders import Nominatim
import folium
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [14]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronoto_neighborhood_postal_code_joined_df['Borough'].unique()),
        toronoto_neighborhood_postal_code_joined_df.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


Get Toronto latitude and longitude

In [15]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Load `Neighborhood` on a map

In [16]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronoto_neighborhood_postal_code_joined_df['Latitude'], toronoto_neighborhood_postal_code_joined_df['Longitude'], toronoto_neighborhood_postal_code_joined_df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

Load foursquare credentials

In [17]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EI12XOLDRJBMYMEYH454EIOPVTVQCAPX2GSWFKDQXUMB2PCE
CLIENT_SECRET:UCP1THAN35F4E50ERSAJL05MMEH4YD1CN22MXMWUHRRIAIFV


Load venues extraction function

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)

        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                  'Neighborhood Latitude',
                  'Neighborhood Longitude',
                  'Venue',
                  'Venue Latitude',
                  'Venue Longitude',
                  'Venue Category']

    return(nearby_venues)

Checking the venues

In [19]:
toronto_venues = getNearbyVenues(names=toronoto_neighborhood_postal_code_joined_df['Neighbourhood'],latitudes=toronoto_neighborhood_postal_code_joined_df['Latitude'],longitudes=toronoto_neighborhood_postal_code_joined_df['Longitude'])
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Brookbanks Pool,43.751389,-79.332184,Pool
2,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
3,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


Adding venues to data frame

In [20]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

Getting the column id of `Neighborhood`

In [21]:
neighborhood_index_toronto_onehot =  np.where(toronto_onehot.columns.values == 'Neighborhood')[0][0]
print("Neighborhood is on column: ",neighborhood_index_toronto_onehot)
toronto_onehot.columns[neighborhood_index_toronto_onehot]

Neighborhood is on column:  192


'Neighborhood'

Moving `Neighborhood` column to first column

In [22]:
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[neighborhood_index_toronto_onehot]] + list(toronto_onehot.columns[:-neighborhood_index_toronto_onehot])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Checking the data frame shape

In [23]:
toronto_onehot.shape

(2138, 83)

Making a grouby of the `Neighborhood`

In [24]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.000000
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,"Willowdale, Willowdale East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000
91,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000
92,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000
93,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.142857


Checking the new data frame shape

In [25]:
toronto_grouped.shape

(95, 83)

Load function to get most common venues

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)

    return row_categories_sorted.index.values[0:num_top_venues]


Getting top 10 venues for the neighborhood nad making a new data frame

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Bagel Shop,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast
1,"Alderwood, Long Branch",Coffee Shop,Curling Ice,Beach,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Bridal Shop,Convenience Store,Curling Ice,Beach,Bakery,Bar,Baseball Field,Baseball Stadium
3,Bayview Village,Chinese Restaurant,Bank,Café,Curling Ice,Beach,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Café,Butcher,American Restaurant,Comfort Food Restaurant,Curling Ice,Basketball Stadium,Bank,Bar,Baseball Field


Setting the number of clusters and generatting the labels

In [28]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 5, 3, 9, 3, 3, 6, 9, 0, 0])

Merging neighborhood venues in toronto data frame

In [29]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronoto_neighborhood_postal_code_joined_df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged_neighborhood_venues_sort_joined = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

#handle neighborhood without venues nor clusters
toronto_merged_neighborhood_venues_sort_joined['Cluster Labels'].fillna(kclusters,inplace=True)
toronto_merged_neighborhood_venues_sort_joined['Cluster Labels']= toronto_merged_neighborhood_venues_sort_joined['Cluster Labels'].astype(pd.Int64Dtype())

toronto_merged_neighborhood_venues_sort_joined.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M3A,North York,Parkwoods,43.753259,-79.329656,0,Curling Ice,Bagel Shop,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast
3,M4A,North York,Victoria Village,43.725882,-79.315572,5,Coffee Shop,Curling Ice,Beach,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3,Coffee Shop,Bakery,Café,Breakfast Spot,Chocolate Shop,Bank,Beer Store,Antique Shop,Art Gallery,Cosmetics Shop
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,3,Clothing Store,Accessories Store,Athletics & Sports,Coffee Shop,Boutique,Airport Gate,Bakery,Bar,Baseball Field,Baseball Stadium
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,5,Coffee Shop,Creperie,Bank,Bar,Beer Bar,Burrito Place,Café,College Auditorium,Curling Ice,Basketball Court


Creating the new map

In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

Adding markers to the map and loading the map

In [31]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_neighborhood_venues_sort_joined['Latitude'], toronto_merged_neighborhood_venues_sort_joined['Longitude'], toronto_merged_neighborhood_venues_sort_joined['Neighbourhood'], toronto_merged_neighborhood_venues_sort_joined['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

Clusters tables

In [32]:
#cluster 0
toronto_merged_neighborhood_venues_sort_joined.loc[
    toronto_merged_neighborhood_venues_sort_joined['Cluster Labels'] == 0,
    toronto_merged_neighborhood_venues_sort_joined.columns[[1] + list(range(5,
    toronto_merged_neighborhood_venues_sort_joined.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,North York,0,Curling Ice,Bagel Shop,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast
9,Scarborough,0,Curling Ice,Bagel Shop,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast
14,North York,0,Bakery,Asian Restaurant,Bed & Breakfast,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach
21,East York,0,Curling Ice,Athletics & Sports,Beer Store,Bus Stop,Beach,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court
23,York,0,Curling Ice,Bagel Shop,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast
27,Scarborough,0,Breakfast Spot,Bank,Bagel Shop,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast
30,East Toronto,0,Curling Ice,Bagel Shop,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast
32,York,0,Curling Ice,Bagel Shop,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast
45,Scarborough,0,Bakery,Bank,Caribbean Restaurant,Athletics & Sports,Beach,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium
46,North York,0,Curling Ice,Bagel Shop,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast


In [33]:
#cluster 1
toronto_merged_neighborhood_venues_sort_joined.loc[
    toronto_merged_neighborhood_venues_sort_joined['Cluster Labels'] == 1,
    toronto_merged_neighborhood_venues_sort_joined.columns[[1] + list(range(5,
    toronto_merged_neighborhood_venues_sort_joined.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Scarborough,1,Bar,Curling Ice,Bagel Shop,Bank,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast


In [34]:
#cluster 2
toronto_merged_neighborhood_venues_sort_joined.loc[
    toronto_merged_neighborhood_venues_sort_joined['Cluster Labels'] == 2,
    toronto_merged_neighborhood_venues_sort_joined.columns[[1] + list(range(5,
    toronto_merged_neighborhood_venues_sort_joined.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,East York,2,Convenience Store,Curling Ice,Bed & Breakfast,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium
98,York,2,Convenience Store,Curling Ice,Bed & Breakfast,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium
100,North York,2,Convenience Store,Curling Ice,Bed & Breakfast,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium


In [35]:
#cluster 3
toronto_merged_neighborhood_venues_sort_joined.loc[
    toronto_merged_neighborhood_venues_sort_joined['Cluster Labels'] == 3,
    toronto_merged_neighborhood_venues_sort_joined.columns[[1] + list(range(5,
    toronto_merged_neighborhood_venues_sort_joined.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Downtown Toronto,3,Coffee Shop,Bakery,Café,Breakfast Spot,Chocolate Shop,Bank,Beer Store,Antique Shop,Art Gallery,Cosmetics Shop
5,North York,3,Clothing Store,Accessories Store,Athletics & Sports,Coffee Shop,Boutique,Airport Gate,Bakery,Bar,Baseball Field,Baseball Stadium
11,North York,3,Coffee Shop,Beer Store,Art Gallery,Café,Caribbean Restaurant,Clothing Store,Athletics & Sports,Chinese Restaurant,Bike Shop,Bed & Breakfast
13,Downtown Toronto,3,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Bubble Tea Shop,Bookstore,Chinese Restaurant,Burger Joint,Bakery,Bank
20,North York,3,Coffee Shop,Beer Store,Art Gallery,Café,Caribbean Restaurant,Clothing Store,Athletics & Sports,Chinese Restaurant,Bike Shop,Bed & Breakfast
22,Downtown Toronto,3,Coffee Shop,Café,Cocktail Bar,American Restaurant,Creperie,Cheese Shop,Cosmetics Shop,Clothing Store,Breakfast Spot,BBQ Joint
26,Etobicoke,3,Coffee Shop,Convenience Store,Beer Store,Café,Curling Ice,Basketball Stadium,Bakery,Bank,Bar,Baseball Field
31,Downtown Toronto,3,Coffee Shop,Cocktail Bar,Cheese Shop,Bakery,Beer Bar,Bistro,Basketball Stadium,Café,Butcher,Clothing Store
39,East York,3,Coffee Shop,Bank,Burger Joint,Breakfast Spot,Brewery,Bike Shop,Beer Store,Bagel Shop,Basketball Stadium,Bar
47,North York,3,Bank,Coffee Shop,Bridal Shop,Convenience Store,Curling Ice,Beach,Bakery,Bar,Baseball Field,Baseball Stadium


In [36]:
#cluster 4
toronto_merged_neighborhood_venues_sort_joined.loc[
    toronto_merged_neighborhood_venues_sort_joined['Cluster Labels'] == 4,
    toronto_merged_neighborhood_venues_sort_joined.columns[[1] + list(range(5,
    toronto_merged_neighborhood_venues_sort_joined.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
89,North York,4,Baseball Field,Curling Ice,Bagel Shop,Bank,Bar,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast
169,Etobicoke,4,Baseball Field,Curling Ice,Bagel Shop,Bank,Bar,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast


In [37]:
#cluster 5
toronto_merged_neighborhood_venues_sort_joined.loc[
    toronto_merged_neighborhood_venues_sort_joined['Cluster Labels'] == 5,
    toronto_merged_neighborhood_venues_sort_joined.columns[[1] + list(range(5,
    toronto_merged_neighborhood_venues_sort_joined.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,North York,5,Coffee Shop,Curling Ice,Beach,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium
6,Downtown Toronto,5,Coffee Shop,Creperie,Bank,Bar,Beer Bar,Burrito Place,Café,College Auditorium,Curling Ice,Basketball Court
36,Scarborough,5,Coffee Shop,Curling Ice,Beach,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium
40,Downtown Toronto,5,Coffee Shop,Café,Bubble Tea Shop,Burger Joint,Bank,Comic Shop,Art Museum,Curling Ice,Basketball Stadium,Bar
63,Scarborough,5,Coffee Shop,Convenience Store,Curling Ice,Beach,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court
86,York,5,Coffee Shop,Convenience Store,Curling Ice,Beach,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court
109,North York,5,Coffee Shop,Curling Ice,Beach,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium
114,Mississauga,5,Coffee Shop,Burrito Place,American Restaurant,Curling Ice,Bed & Breakfast,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court
151,Etobicoke,5,Coffee Shop,Curling Ice,Beach,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium


In [38]:
#cluster 6
toronto_merged_neighborhood_venues_sort_joined.loc[
    toronto_merged_neighborhood_venues_sort_joined['Cluster Labels'] == 6,
    toronto_merged_neighborhood_venues_sort_joined.columns[[1] + list(range(5,
    toronto_merged_neighborhood_venues_sort_joined.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
90,Scarborough,6,Café,College Stadium,Curling Ice,Beach,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium


In [39]:
#cluster 7
toronto_merged_neighborhood_venues_sort_joined.loc[
    toronto_merged_neighborhood_venues_sort_joined['Cluster Labels'] == 7,
    toronto_merged_neighborhood_venues_sort_joined.columns[[1] + list(range(5,
    toronto_merged_neighborhood_venues_sort_joined.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
77,North York,7,Bakery,Construction & Landscaping,Basketball Court,Curling Ice,Bed & Breakfast,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium


In [40]:
#cluster 8
toronto_merged_neighborhood_venues_sort_joined.loc[
    toronto_merged_neighborhood_venues_sort_joined['Cluster Labels'] == 8,
    toronto_merged_neighborhood_venues_sort_joined.columns[[1] + list(range(5,
    toronto_merged_neighborhood_venues_sort_joined.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
72,Scarborough,8,Bakery,Bus Line,Bus Station,Bed & Breakfast,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium
93,Central Toronto,8,Business Service,Bus Line,Curling Ice,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach
95,York,8,Breakfast Spot,Caribbean Restaurant,Bus Line,Bed & Breakfast,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium
103,Central Toronto,8,Bus Line,Curling Ice,Bed & Breakfast,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach
116,Etobicoke,8,Bus Line,Curling Ice,Bed & Breakfast,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach


In [41]:
#cluster 9
toronto_merged_neighborhood_venues_sort_joined.loc[
    toronto_merged_neighborhood_venues_sort_joined['Cluster Labels'] == 9,
    toronto_merged_neighborhood_venues_sort_joined.columns[[1] + list(range(5,
    toronto_merged_neighborhood_venues_sort_joined.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,East York,9,Breakfast Spot,Bank,Athletics & Sports,Café,Beach,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium
41,Downtown Toronto,9,Café,Baby Store,Candy Store,Coffee Shop,Athletics & Sports,Bagel Shop,Bank,Bar,Baseball Field,Baseball Stadium
50,West Toronto,9,Bakery,Brewery,Bank,Bar,Café,Bed & Breakfast,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium
64,North York,9,Chinese Restaurant,Bank,Café,Curling Ice,Beach,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium
68,West Toronto,9,Café,Breakfast Spot,Coffee Shop,Bakery,Climbing Gym,Bar,Convenience Store,Burrito Place,Basketball Court,Bank
104,West Toronto,9,Café,Bookstore,Bar,Cajun / Creole Restaurant,Arts & Crafts Store,Antique Shop,Bakery,Curling Ice,Basketball Stadium,Baseball Field
121,Downtown Toronto,9,Café,Bookstore,Bakery,Bar,Bank,Beer Store,Beer Bar,College Arts Building,Comfort Food Restaurant,College Gym
142,Etobicoke,9,Café,Bakery,American Restaurant,Bed & Breakfast,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium


In [42]:
#cluster 10
toronto_merged_neighborhood_venues_sort_joined.loc[
    toronto_merged_neighborhood_venues_sort_joined['Cluster Labels'] == 10,
    toronto_merged_neighborhood_venues_sort_joined.columns[[1] + list(range(5,
    toronto_merged_neighborhood_venues_sort_joined.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Etobicoke,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,Etobicoke,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,North York,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153,Scarborough,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
